<a href="https://colab.research.google.com/github/pytholabsbot1/cats_dogs_classifier/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [0]:
!export KAGGLE_USERNAME=vaqasam
!export KAGGLE_KEY=2ec5d91c66a678bc314920e84c687f3b

In [8]:
!ls /root/.kaggle

open('/root/.kaggle/kaggle.json','w').write('{"username":"vaqasam","key":"2ec5d91c66a678bc314920e84c687f3b"}')

63

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [12]:
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 25.5MB/s]
 98% 267M/271M [00:01<00:00, 157MB/s]
100% 271M/271M [00:01<00:00, 154MB/s]
100% 542M/543M [00:03<00:00, 153MB/s]
100% 543M/543M [00:03<00:00, 147MB/s]


In [15]:

!mkdir /content/train
!mkdir /content/test

!ls /content

kaggle.json  sampleSubmission.csv  test1.zip  train.zip
sample_data  test		   train


In [0]:
!unzip train.zip -d /content/train

In [0]:
!mkdir /content/train/train/new_test/cats

In [0]:
import os,shutil

for i in os.listdir('/content/train/train'):
  try:
    if('cat' in i):
      shutil.copyfile('/content/train/train/'+i,'/content/train/train/new/cats/'+i)

    else:
      shutil.copyfile('/content/train/train/'+i,'/content/train/train/new/dogs/'+i)
      
  except:
    print('error')
  print(i)

In [0]:
import os,shutil

for i in range(500):
  os.system('sudo mv /content/train/train/new/dogs/{} /content/train/train/new_test/dogs/{}'.format(d[i],d[i]))
  print(i)

In [0]:
!ls /content/train/train/new_test/dogs

In [32]:
import os,shutil

c = [i for i in os.listdir('/content/train/train/new/cats')]
d = [i for i in os.listdir('/content/train/train/new/dogs')]

print('length dogs,cats :{} , {}'.format(len(d),len(c)))


length dogs,cats :12500 , 12500


In [0]:
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/train/train/new/',
                                                 target_size = (64, 64),
                                                 batch_size = 100,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/train/train/new_test/',
                                            target_size = (64, 64),
                                            batch_size = 100,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 120,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 5)

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/25
120/120 [==============================] - 114s 950ms/step - loss: 0.6880 - acc: 0.5623 - val_loss: 0.6312 - val_acc: 0.6280
Epoch 2/25
120/120 [==============================] - 111s 925ms/step - loss: 0.6059 - acc: 0.6713 - val_loss: 0.5892 - val_acc: 0.6900
Epoch 3/25
120/120 [==============================] - 111s 927ms/step - loss: 0.5645 - acc: 0.7127 - val_loss: 0.5646 - val_acc: 0.7100
Epoch 4/25
  4/120 [>.............................] - ETA: 1:53 - loss: 0.5403 - acc: 0.7275